In [352]:
import pandas as pd 
import numpy as np
import nltk
from nltk.tag import StanfordNERTagger
os.environ.get('CLASSPATH')
t = StanfordNERTagger('english.all.3class.distsim.crf.ser.gz')
import os
os.environ['CLASSPATH'] ='/home/rudra/stanford-ner-2016-10-31/stanford-ner.jar'
os.environ['STANFORD_MODELS'] = '/home/rudra/stanford-ner-2016-10-31/classifiers'
import string 
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [353]:
#read train data
train = pd.read_csv('/home/rudra/Quora_Pair/Data/train.csv')



# 

1. to english 
2. tokenize
3. cleaning ( stop words, punc ) 
4. function question type ( feature ) 
5. POS tagg 
6. bi-grams + tri-grams
7. count of unique words 
8. intersecting words
9. Non-inter"" words
10. equation in latex ? 
11. Keyword or wordcloud 
12. Sentiment 


MOdels Experimentation : 
     LR
     SVM 
     Naive Bayes 
     
10. W2V

In [354]:

stopwords = set(stopwords.words('english'))


In [355]:
#q1_tok = [[(w.lower()) for w in word_tokenize(t.decode('utf-8')) if (w not in stopwords and w not in string.punctuation)] for t in train["question1"]]
#tokenizing question1
q1_tok = []

for t in train["question1"]:
    temp = []
    if type(t) != float : 
        for w in word_tokenize(t.decode('utf-8')):
            if (w not in stopwords and w not in string.punctuation):
                temp.append(w.lower())
    q1_tok.append(temp) if temp else q1_tok.append([""]) 

train['tokenize_question1'] = q1_tok

In [356]:
#tokenizing q2
q2_tok = []

for t in train["question2"]:
    temp = []
    if type(t) != float : 
        for w in word_tokenize(t.decode('utf-8')):
            if (w not in stopwords and w not in string.punctuation):
                temp.append(w.lower())
    q2_tok.append(temp) if temp else q2_tok.append([""]) 
train['tokenize_question2'] = q2_tok

In [389]:
df = pd.read_pickle("/home/rudra/Quora_Pair/Data.pkl")


In [359]:
#Function to get the organization person and location (Failed attempt to cluster)
def Org_Per_Loc(sent) : 
    loc,per,org = [],[],[]
    for (w,i) in t.tag(sent): #word_tokenize(sent.decode('utf-8'))
        if i == 'LOCATION' : 
            loc.append(w)
        elif i == 'PERSON':
            per.append(w)
        elif i == 'ORGANIZATION':
            org.append(w)

    return (loc,per,org)

In [390]:
#generate ngrams depending on the input of q1 and q1  and the 
from nltk import ngrams
    
def ngrams_(sent):
    res = []
    for n in xrange(2,5): 
        temp = []
        temp_gram = set(ngrams(sent, n))
        for i in temp_gram: 
            temp.append(i)
        
        res.append(temp)
    return ((res[0]),(res[1]),(res[2]))

In [391]:
#counting the similarity scores of the 2 questions
def similarity(sent1,sent2): #returns similarity scores 
    count_bi,count_tri,count_quad = 0.0,0.0,0.0
    bias2,bias3,bias4=1.0,1.0,1.0 # further testing

    
    bi_temp1,tri_temp1,quad_temp1 = ngrams_(sent1)
    bi_temp2,tri_temp2,quad_temp2 = ngrams_(sent2)


    try : 
        count_bi = (bias2*len(set(bi_temp1) & set(bi_temp2)))/float(len(set(bi_temp1) | set(bi_temp2)))
    except ZeroDivisionError as err:
        count_bi = 0
    try :
        count_tri = (bias3*len(set(tri_temp1) & set(tri_temp2)))/float(len(set(tri_temp1) | set(tri_temp2)))
    except ZeroDivisionError as err:
        count_tri = 0
        
    try :
        count_quad = (bias4*len(set(quad_temp1) & set(quad_temp2)))/float(len(set(quad_temp1) | set(quad_temp2)))
    except ZeroDivisionError as err:
        count_quad = 0
        
    return (count_bi,count_tri,count_quad)

In [392]:
#gives the similarity scores 
c2,c3,c4 = [],[],[]

i = 0
#print 
while i < (df.shape[0]):
    sent1= df["tokenize_question1"][i]
    sent2 =df["tokenize_question2"][i]   

    if i %100000== 0 :
        print str(i) + "iterations done"
    
    (t2,t3,t4) = similarity(sent1,sent2)
    c2.append(t2)
    c3.append(t3)
    c4.append(t4)
    i+=1

0iterations done
100000iterations done
200000iterations done
300000iterations done
400000iterations done


In [393]:
#counting inclusion and exclusion of sent1 and sent2 
def a_not_in_b(sent1,sent2):
    anotb,bnota,aandb = 0.0,0.0,0.0
    q_set1 = set(sent1)
    q_set2 = set(sent2)
    a = float(len(q_set1 - q_set2))
    b = float(len(q_set2 - q_set1))
    ab= float(len(q_set1 & q_set2))
    anotb = a/float(len(q_set1))
    bnota = b/float(len(q_set2))
    aandb = ab/float(len(q_set1 | q_set2))
    
    return (anotb,bnota,aandb,a,b,ab)
    

In [394]:
anotb,bnota,aandb = [],[],[]
acount,bcount,abcount = [],[],[]
i = 0
while i < df.shape[0]:
    sent1 = df["tokenize_question1"][i]
    sent2 = df["tokenize_question2"][i] 
    if i %100000 == 0 :
        print str(i) + "iterations done"
    (a,b,ab,a_,b_,ab_) = a_not_in_b(sent1,sent2)
    anotb.append(a)
    bnota.append(b)
    aandb.append(ab)
    acount.append(a_)
    bcount.append(b_)
    abcount.append(ab_)
    i+=1

0iterations done
100000iterations done
200000iterations done
300000iterations done
400000iterations done


In [395]:
df["bi_gram_score"] = c2
df["tri_gram_score"] = c3
df["quad_gram_score"] = c4

    

In [396]:
df["InQ1NotinQ2_words"] = acount
df["NotinQ1InQ2_words"] = bcount
df["Intersect_Q1Q2_clean"] = abcount

df["in_q1_not_in_q2"] = anotb
df["in_q2_not_in_q1"] = bnota
df["intersection_q1_q2"] = aandb

df.head()

,id,qid1,qid2,question1,question2,is_duplicate,tokenize_question1,tokenize_question2,bi_gram_score,tri_gram_score,quad_gram_score,InQ1NotinQ2_words,NotinQ1InQ2_words,Intersect_Q1Q2_clean,in_q1_not_in_q2,in_q2_not_in_q1,intersection_q1_q2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[what, step, step, guide, invest, share, marke...","[what, step, step, guide, invest, share, market]",0.857143,0.833333,0.8,1.0,0.0,6.0,0.142857,0.0,0.857143
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[what, story, kohinoor, koh-i-noor, diamond]","[what, would, happen, indian, government, stol...",0.181818,0.100000,0.0,1.0,6.0,4.0,0.200000,0.6,0.363636
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[how, i, increase, speed, internet, connection...","[how, internet, speed, increased, hacking, dns]",0.000000,0.000000,0.0,5.0,3.0,3.0,0.625000,0.5,0.272727
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[why, i, mentally, lonely, how, i, solve]","[find, remainder, math, 23^, 24, /math, divide...",0.000000,0.000000,0.0,6.0,8.0,0.0,1.000000,1.0,0.000000
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[which, one, dissolve, water, quikly, sugar, s...","[which, fish, would, survive, salt, water]",0.000000,0.000000,0.0,8.0,3.0,3.0,0.727273,0.5,0.214286


In [397]:
df.to_pickle("/home/rudra/Quora_Pair/Data.pkl")

In [254]:
from sklearn.naive_bayes import GaussianNB


In [267]:

# Base Baysian Model

X = df[['bi_gram_score','tri_gram_score','quad_gram_score']]
clf  = GaussianNB()
clf.fit(X,df['is_duplicate'])

GaussianNB(priors=None)

In [332]:
q1_tok = []
for t in test["question1"]:
    temp = []
    if type(t) != float : 
        for w in word_tokenize(t.decode('utf-8')):
            if (w not in stopwords and w not in string.punctuation):
                temp.append(w.lower())
    q1_tok.append(temp) if temp else q1_tok.append([""]) 

q2_tok = []
for t in test["question2"]:
    temp = []
    if type(t) != float : 
        for w in word_tokenize(t.decode('utf-8')):
            if (w not in stopwords and w not in string.punctuation):
                temp.append(w.lower())
    q2_tok.append(temp) if temp else q2_tok.append([""]) 


KeyboardInterrupt: 

In [333]:
test = pd.read_pickle("/home/rudra/Quora_Pair/Data_test.pkl")

In [288]:

c2,c3,c4 = [],[],[]
i = 0
#print 
while i < (test.shape[0]):
    sent1= test["tokenize_question1"][i]
    sent2 =test["tokenize_question2"][i]   

    if i %1000 == 0 :
        print str(i) + "iterations done"
    
    (t2,t3,t4) = similarity(sent1,sent2)
    c2.append(t2)
    c3.append(t3)
    c4.append(t4)
    i+=1

0iterations done
1000iterations done
2000iterations done
3000iterations done
4000iterations done
5000iterations done
6000iterations done
7000iterations done
8000iterations done
9000iterations done
10000iterations done
11000iterations done
12000iterations done
13000iterations done
14000iterations done
15000iterations done
16000iterations done
17000iterations done
18000iterations done
19000iterations done
20000iterations done
21000iterations done
22000iterations done
23000iterations done
24000iterations done
25000iterations done
26000iterations done
27000iterations done
28000iterations done
29000iterations done
30000iterations done
31000iterations done
32000iterations done
33000iterations done
34000iterations done
35000iterations done
36000iterations done
37000iterations done
38000iterations done
39000iterations done
40000iterations done
41000iterations done
42000iterations done
43000iterations done
44000iterations done
45000iterations done
46000iterations done
47000iterations done
48000

383000iterations done
384000iterations done
385000iterations done
386000iterations done
387000iterations done
388000iterations done
389000iterations done
390000iterations done
391000iterations done
392000iterations done
393000iterations done
394000iterations done
395000iterations done
396000iterations done
397000iterations done
398000iterations done
399000iterations done
400000iterations done
401000iterations done
402000iterations done
403000iterations done
404000iterations done
405000iterations done
406000iterations done
407000iterations done
408000iterations done
409000iterations done
410000iterations done
411000iterations done
412000iterations done
413000iterations done
414000iterations done
415000iterations done
416000iterations done
417000iterations done
418000iterations done
419000iterations done
420000iterations done
421000iterations done
422000iterations done
423000iterations done
424000iterations done
425000iterations done
426000iterations done
427000iterations done
428000iter

1122000iterations done
1123000iterations done
1124000iterations done
1125000iterations done
1126000iterations done
1127000iterations done
1128000iterations done
1129000iterations done
1130000iterations done
1131000iterations done
1132000iterations done
1133000iterations done
1134000iterations done
1135000iterations done
1136000iterations done
1137000iterations done
1138000iterations done
1139000iterations done
1140000iterations done
1141000iterations done
1142000iterations done
1143000iterations done
1144000iterations done
1145000iterations done
1146000iterations done
1147000iterations done
1148000iterations done
1149000iterations done
1150000iterations done
1151000iterations done
1152000iterations done
1153000iterations done
1154000iterations done
1155000iterations done
1156000iterations done
1157000iterations done
1158000iterations done
1159000iterations done
1160000iterations done
1161000iterations done
1162000iterations done
1163000iterations done
1164000iterations done
1165000iter

1479000iterations done
1480000iterations done
1481000iterations done
1482000iterations done
1483000iterations done
1484000iterations done
1485000iterations done
1486000iterations done
1487000iterations done
1488000iterations done
1489000iterations done
1490000iterations done
1491000iterations done
1492000iterations done
1493000iterations done
1494000iterations done
1495000iterations done
1496000iterations done
1497000iterations done
1498000iterations done
1499000iterations done
1500000iterations done
1501000iterations done
1502000iterations done
1503000iterations done
1504000iterations done
1505000iterations done
1506000iterations done
1507000iterations done
1508000iterations done
1509000iterations done
1510000iterations done
1511000iterations done
1512000iterations done
1513000iterations done
1514000iterations done
1515000iterations done
1516000iterations done
1517000iterations done
1518000iterations done
1519000iterations done
1520000iterations done
1521000iterations done
1522000iter

1838000iterations done
1839000iterations done
1840000iterations done
1841000iterations done
1842000iterations done
1843000iterations done
1844000iterations done
1845000iterations done
1846000iterations done
1847000iterations done
1848000iterations done
1849000iterations done
1850000iterations done
1851000iterations done
1852000iterations done
1853000iterations done
1854000iterations done
1855000iterations done
1856000iterations done
1857000iterations done
1858000iterations done
1859000iterations done
1860000iterations done
1861000iterations done
1862000iterations done
1863000iterations done
1864000iterations done
1865000iterations done
1866000iterations done
1867000iterations done
1868000iterations done
1869000iterations done
1870000iterations done
1871000iterations done
1872000iterations done
1873000iterations done
1874000iterations done
1875000iterations done
1876000iterations done
1877000iterations done
1878000iterations done
1879000iterations done
1880000iterations done
1881000iter

2196000iterations done
2197000iterations done
2198000iterations done
2199000iterations done
2200000iterations done
2201000iterations done
2202000iterations done
2203000iterations done
2204000iterations done
2205000iterations done
2206000iterations done
2207000iterations done
2208000iterations done
2209000iterations done
2210000iterations done
2211000iterations done
2212000iterations done
2213000iterations done
2214000iterations done
2215000iterations done
2216000iterations done
2217000iterations done
2218000iterations done
2219000iterations done
2220000iterations done
2221000iterations done
2222000iterations done
2223000iterations done
2224000iterations done
2225000iterations done
2226000iterations done
2227000iterations done
2228000iterations done
2229000iterations done
2230000iterations done
2231000iterations done
2232000iterations done
2233000iterations done
2234000iterations done
2235000iterations done
2236000iterations done
2237000iterations done
2238000iterations done
2239000iter

In [287]:
test['tokenize_question1'] = q1_tok
test['tokenize_question2'] = q2_tok

,test_id,question1,question2,tokenize_question1,tokenize_question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,"[how, surface, pro, 4, compare, ipad, pro]","[why, microsoft, choose, core, m3, core, i3, h..."
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,"[should, i, hair, transplant, age, 24, how, mu...","[how, much, cost, hair, transplant, require]"
2,2,What but is the best way to send money from Ch...,What you send money to China?,"[what, best, way, send, money, china, us]","[what, send, money, china]"
3,3,Which food not emulsifiers?,What foods fibre?,"[which, food, emulsifiers]","[what, foods, fibre]"
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,"[how, ``, aberystwyth, '', start, reading]","[how, i, start, reading]"


In [315]:
#gives the similarity scores 
c2,c3,c4 = [],[],[]

i = 0
#print 
while i < (test.shape[0]):
    sent1 = test["tokenize_question1"][i]
    sent2 = test["tokenize_question2"][i]   

    if i %100000 == 0 :
        print str(i) + "iterations done"
    
    (t2,t3,t4) = similarity(sent1,sent2)
    c2.append(t2)
    c3.append(t3)
    c4.append(t4)
    i+=1

0iterations done
1000iterations done
2000iterations done
3000iterations done
4000iterations done
5000iterations done
6000iterations done
7000iterations done
8000iterations done
9000iterations done
10000iterations done
11000iterations done
12000iterations done
13000iterations done
14000iterations done
15000iterations done
16000iterations done
17000iterations done
18000iterations done
19000iterations done
20000iterations done
21000iterations done
22000iterations done
23000iterations done
24000iterations done
25000iterations done
26000iterations done
27000iterations done
28000iterations done
29000iterations done
30000iterations done
31000iterations done
32000iterations done
33000iterations done
34000iterations done
35000iterations done
36000iterations done
37000iterations done
38000iterations done
39000iterations done
40000iterations done
41000iterations done
42000iterations done
43000iterations done
44000iterations done
45000iterations done
46000iterations done
47000iterations done
48000

379000iterations done
380000iterations done
381000iterations done
382000iterations done
383000iterations done
384000iterations done
385000iterations done
386000iterations done
387000iterations done
388000iterations done
389000iterations done
390000iterations done
391000iterations done
392000iterations done
393000iterations done
394000iterations done
395000iterations done
396000iterations done
397000iterations done
398000iterations done
399000iterations done
400000iterations done
401000iterations done
402000iterations done
403000iterations done
404000iterations done
405000iterations done
406000iterations done
407000iterations done
408000iterations done
409000iterations done
410000iterations done
411000iterations done
412000iterations done
413000iterations done
414000iterations done
415000iterations done
416000iterations done
417000iterations done
418000iterations done
419000iterations done
420000iterations done
421000iterations done
422000iterations done
423000iterations done
424000iter

755000iterations done
756000iterations done
757000iterations done
758000iterations done
759000iterations done
760000iterations done
761000iterations done
762000iterations done
763000iterations done
764000iterations done
765000iterations done
766000iterations done
767000iterations done
768000iterations done
769000iterations done
770000iterations done
771000iterations done
772000iterations done
773000iterations done
774000iterations done
775000iterations done
776000iterations done
777000iterations done
778000iterations done
779000iterations done
780000iterations done
781000iterations done
782000iterations done
783000iterations done
784000iterations done
785000iterations done
786000iterations done
787000iterations done
788000iterations done
789000iterations done
790000iterations done
791000iterations done
792000iterations done
793000iterations done
794000iterations done
795000iterations done
796000iterations done
797000iterations done
798000iterations done
799000iterations done
800000iter

1123000iterations done
1124000iterations done
1125000iterations done
1126000iterations done
1127000iterations done
1128000iterations done
1129000iterations done
1130000iterations done
1131000iterations done
1132000iterations done
1133000iterations done
1134000iterations done
1135000iterations done
1136000iterations done
1137000iterations done
1138000iterations done
1139000iterations done
1140000iterations done
1141000iterations done
1142000iterations done
1143000iterations done
1144000iterations done
1145000iterations done
1146000iterations done
1147000iterations done
1148000iterations done
1149000iterations done
1150000iterations done
1151000iterations done
1152000iterations done
1153000iterations done
1154000iterations done
1155000iterations done
1156000iterations done
1157000iterations done
1158000iterations done
1159000iterations done
1160000iterations done
1161000iterations done
1162000iterations done
1163000iterations done
1164000iterations done
1165000iterations done
1166000iter

1837000iterations done
1838000iterations done
1839000iterations done
1840000iterations done
1841000iterations done
1842000iterations done
1843000iterations done
1844000iterations done
1845000iterations done
1846000iterations done
1847000iterations done
1848000iterations done
1849000iterations done
1850000iterations done
1851000iterations done
1852000iterations done
1853000iterations done
1854000iterations done
1855000iterations done
1856000iterations done
1857000iterations done
1858000iterations done
1859000iterations done
1860000iterations done
1861000iterations done
1862000iterations done
1863000iterations done
1864000iterations done
1865000iterations done
1866000iterations done
1867000iterations done
1868000iterations done
1869000iterations done
1870000iterations done
1871000iterations done
1872000iterations done
1873000iterations done
1874000iterations done
1875000iterations done
1876000iterations done
1877000iterations done
1878000iterations done
1879000iterations done
1880000iter

2196000iterations done
2197000iterations done
2198000iterations done
2199000iterations done
2200000iterations done
2201000iterations done
2202000iterations done
2203000iterations done
2204000iterations done
2205000iterations done
2206000iterations done
2207000iterations done
2208000iterations done
2209000iterations done
2210000iterations done
2211000iterations done
2212000iterations done
2213000iterations done
2214000iterations done
2215000iterations done
2216000iterations done
2217000iterations done
2218000iterations done
2219000iterations done
2220000iterations done
2221000iterations done
2222000iterations done
2223000iterations done
2224000iterations done
2225000iterations done
2226000iterations done
2227000iterations done
2228000iterations done
2229000iterations done
2230000iterations done
2231000iterations done
2232000iterations done
2233000iterations done
2234000iterations done
2235000iterations done
2236000iterations done
2237000iterations done
2238000iterations done
2239000iter

In [289]:
test["bi_gram_score"] = c2
test["tri_gram_score"] = c3
test["quad_gram_score"] = c4

In [336]:
anotb,bnota,aandb = [],[],[]
acount,bcount,abcount = [],[],[]
i = 0
while i < test.shape[0]:
    sent1 = test["tokenize_question1"][i]
    sent2 = test["tokenize_question2"][i] 
    if i %10000 == 0 :
        print str(i) + "iterations done"
    (a,b,ab,a_,b_,ab_) = a_not_in_b(sent1,sent2)
    anotb.append(a)
    bnota.append(b)
    aandb.append(ab)
    acount.append(a_)
    bcount.append(b_)
    abcount.append(ab_)
    i+=1
    

0iterations done
10000iterations done
20000iterations done
30000iterations done
40000iterations done
50000iterations done
60000iterations done
70000iterations done
80000iterations done
90000iterations done
100000iterations done
110000iterations done
120000iterations done
130000iterations done
140000iterations done
150000iterations done
160000iterations done
170000iterations done
180000iterations done
190000iterations done
200000iterations done
210000iterations done
220000iterations done
230000iterations done
240000iterations done
250000iterations done
260000iterations done
270000iterations done
280000iterations done
290000iterations done
300000iterations done
310000iterations done
320000iterations done
330000iterations done
340000iterations done
350000iterations done
360000iterations done
370000iterations done
380000iterations done
390000iterations done
400000iterations done
410000iterations done
420000iterations done
430000iterations done
440000iterations done
450000iterations done
46

In [337]:
test["InQ1NotinQ2_words"] = acount
test["NotinQ1InQ2_words"] = bcount
test["Intersect_Q1Q2_clean"] = abcount
test["in_q1_not_in_q2"] = anotb
test["in_q2_not_in_q1"] = bnota
test["intersection_q1_q2"] = aandb

,test_id,question1,question2,tokenize_question1,tokenize_question2,bi_gram_score,tri_gram_score,quad_gram_score,InQ1NotinQ2_words,NotinQ1InQ2_words,Intersect_Q1Q2_clean,in_q1_not_in_q2,in_q2_not_in_q1,intersection_q1_q2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,"[how, surface, pro, 4, compare, ipad, pro]","[why, microsoft, choose, core, m3, core, i3, h...",0.142857,0.076923,0.0,3.0,7.0,3.0,0.500000,0.700000,0.230769
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,"[should, i, hair, transplant, age, 24, how, mu...","[how, much, cost, hair, transplant, require]",0.166667,0.000000,0.0,5.0,1.0,5.0,0.500000,0.166667,0.454545
2,2,What but is the best way to send money from Ch...,What you send money to China?,"[what, best, way, send, money, china, us]","[what, send, money, china]",0.285714,0.166667,0.0,3.0,0.0,4.0,0.428571,0.000000,0.571429
3,3,Which food not emulsifiers?,What foods fibre?,"[which, food, emulsifiers]","[what, foods, fibre]",0.000000,0.000000,0.0,3.0,3.0,0.0,1.000000,1.000000,0.000000
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,"[how, ``, aberystwyth, '', start, reading]","[how, i, start, reading]",0.142857,0.000000,0.0,3.0,1.0,3.0,0.500000,0.250000,0.428571


In [341]:
del test["question1"]
del test["question2"]
test.head()

,test_id,tokenize_question1,tokenize_question2,bi_gram_score,tri_gram_score,quad_gram_score,InQ1NotinQ2_words,NotinQ1InQ2_words,Intersect_Q1Q2_clean,in_q1_not_in_q2,in_q2_not_in_q1,intersection_q1_q2
0,0,"[how, surface, pro, 4, compare, ipad, pro]","[why, microsoft, choose, core, m3, core, i3, h...",0.142857,0.076923,0.0,3.0,7.0,3.0,0.500000,0.700000,0.230769
1,1,"[should, i, hair, transplant, age, 24, how, mu...","[how, much, cost, hair, transplant, require]",0.166667,0.000000,0.0,5.0,1.0,5.0,0.500000,0.166667,0.454545
2,2,"[what, best, way, send, money, china, us]","[what, send, money, china]",0.285714,0.166667,0.0,3.0,0.0,4.0,0.428571,0.000000,0.571429
3,3,"[which, food, emulsifiers]","[what, foods, fibre]",0.000000,0.000000,0.0,3.0,3.0,0.0,1.000000,1.000000,0.000000
4,4,"[how, ``, aberystwyth, '', start, reading]","[how, i, start, reading]",0.142857,0.000000,0.0,3.0,1.0,3.0,0.500000,0.250000,0.428571


In [347]:
test_features= test[['test_id','bi_gram_score', 'tri_gram_score','quad_gram_score','InQ1NotinQ2_words', 
              'NotinQ1InQ2_words', 'Intersect_Q1Q2_clean', 'in_q1_not_in_q2', 'in_q2_not_in_q1', 'intersection_q1_q2']]

In [349]:
test_features.to_pickle("/home/rudra/Quora_Pair/Data_test.pkl")

In [351]:
test_features.head()

,test_id,bi_gram_score,tri_gram_score,quad_gram_score,InQ1NotinQ2_words,NotinQ1InQ2_words,Intersect_Q1Q2_clean,in_q1_not_in_q2,in_q2_not_in_q1,intersection_q1_q2
0,0,0.142857,0.076923,0.0,3.0,7.0,3.0,0.500000,0.700000,0.230769
1,1,0.166667,0.000000,0.0,5.0,1.0,5.0,0.500000,0.166667,0.454545
2,2,0.285714,0.166667,0.0,3.0,0.0,4.0,0.428571,0.000000,0.571429
3,3,0.000000,0.000000,0.0,3.0,3.0,0.0,1.000000,1.000000,0.000000
4,4,0.142857,0.000000,0.0,3.0,1.0,3.0,0.500000,0.250000,0.428571
